In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils


Using TensorFlow backend.
/home/dhama/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
model=Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32,32,3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))


model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(12, activation='softmax'))
# Compile model
epochs = 5
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 128)       73856     
__________

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   vertical_flip=False,
                                   height_shift_range=0.1,
                                   width_shift_range=0.1,
                                   rotation_range=15,
                                  )
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('train/',
                                                 target_size = (32, 32),
                                                 batch_size = 50,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('cross/',
                                            target_size = (32, 32),
                                            batch_size = 10,
                                            class_mode = 'categorical')

model.fit_generator(training_set,
                         steps_per_epoch = 4038,
                         epochs = 10,
                         validation_data = test_set,
                         validation_steps = 712)

Found 4038 images belonging to 12 classes.
Found 712 images belonging to 12 classes.
Epoch 1/10
4038/4038 [==============================] - 6594s 2s/step - loss: 1.1154 - acc: 0.6041 - val_loss: 0.5642 - val_acc: 0.7970
Epoch 2/10
4038/4038 [==============================] - 3862s 956ms/step - loss: 0.5797 - acc: 0.7935 - val_loss: 0.3923 - val_acc: 0.8621
Epoch 3/10
4038/4038 [==============================] - 17355s 4s/step - loss: 0.4476 - acc: 0.8382 - val_loss: 0.3300 - val_acc: 0.8881
Epoch 4/10
4038/4038 [==============================] - 3741s 927ms/step - loss: 0.3812 - acc: 0.8611 - val_loss: 0.3056 - val_acc: 0.8959
Epoch 5/10
4038/4038 [==============================] - 6873s 2s/step - loss: 0.3391 - acc: 0.8748 - val_loss: 0.2844 - val_acc: 0.9028
Epoch 6/10
4038/4038 [==============================] - 3693s 915ms/step - loss: 0.3099 - acc: 0.8851 - val_loss: 0.2782 - val_acc: 0.9065
Epoch 7/10
4038/4038 [==============================] - 3865s 957ms/step - loss: 0.2890 -

In [4]:
from keras.preprocessing.image import ImageDataGenerator

In [6]:
model.save('plantnew.h5')

In [7]:
from keras.models import load_model

In [8]:
model=load_model('plantnew.h5')

In [9]:
import glob
test_list=glob.glob("/home/dhama/Untitled Folder/Convolutional_Neural_Networks/Plant Seedlings Classification/test/*.png")

In [18]:
a='Black-grass'
b='Charlock'
c='Cleavers'
d='Common Chickweed'
e='Common wheat'
f='Fat Hen'
g='Loose Silky-bent'
h='Maize'
p='Scentless Mayweed'
q='Shepherds Purse'
k='Small-flowered Cranesbill'
o='Sugar beet'
ans=[]
import numpy as np
from keras.preprocessing import image
for i in range(len(test_list)):
    test_image = image.load_img(test_list[i], target_size = (32, 32))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(test_image)
    
    if result[0][0]>.5:
        ans.append(a)
    if result[0][1]>.5:
        ans.append(b)
    if result[0][2]>.5:
        ans.append(c)
    if result[0][3]>.8:
        ans.append(d)
    if result[0][3]>0.5 and result[0][3]<.8:
        ans.append(p)
    if result[0][4]>.5:
        ans.apppend(e)
    if result[0][5]>.5:
        ans.append(f)
    if result[0][6]>.5:
        ans.append(g)
    if result[0][7]>.5:
        ans.append(h)
    if result[0][8]>.5:
        ans.append(p)
    if result[0][9]>.5:
        ans.append(q)
    if result[0][10]>.5:
        ans.append(k)
    if result[0][11]>.5:
        ans.append(o)
len(ans)

794

In [19]:
z=test_list

In [20]:
final=[]
for x in range(len(z)):
    final.append(z[x][94:])
final

['f4caf74f9.png',
 '0bf7bfb05.png',
 'd2f422ccb.png',
 'd0cdc768f.png',
 '756dd5070.png',
 '61dd2cdc5.png',
 '67e185673.png',
 'dd5ec63d9.png',
 '8d6acbe9b.png',
 '699d3c707.png',
 '4823c3ffa.png',
 'c85ef220d.png',
 'da5255450.png',
 '5b63dcc21.png',
 '8faadb6a8.png',
 '8a30b2de3.png',
 '39b740f7e.png',
 'a9d2eab61.png',
 '116b136de.png',
 'ff65bc002.png',
 'c7051c902.png',
 '1758a1baf.png',
 '4bcdaa5e2.png',
 'd3331e071.png',
 'fb022edf9.png',
 'eec1079a1.png',
 'cec5bf198.png',
 'f8318faf1.png',
 '7beb2766f.png',
 'bffc08672.png',
 '060f1dc84.png',
 'fc6f686fb.png',
 '897e5a8de.png',
 'bf66b9cd2.png',
 'a006a475c.png',
 'b47691c08.png',
 'cd6adba97.png',
 'e98e5d1d5.png',
 '8ab8a958b.png',
 '8874bba69.png',
 '3f826b318.png',
 'c7b07431e.png',
 'a93f940d6.png',
 '38c054379.png',
 'f1e87cba7.png',
 '3dd52bd2a.png',
 '34dd57ca9.png',
 '122913909.png',
 '17529c555.png',
 '3d65168c2.png',
 '8301b0547.png',
 '5e6a237f2.png',
 '98da6ef4e.png',
 'adb7a032c.png',
 '2e86f1085.png',
 '60fea2ef

In [21]:
ans

['Maize',
 'Loose Silky-bent',
 'Common Chickweed',
 'Loose Silky-bent',
 'Common Chickweed',
 'Loose Silky-bent',
 'Shepherds Purse',
 'Common Chickweed',
 'Common Chickweed',
 'Common Chickweed',
 'Common Chickweed',
 'Common Chickweed',
 'Common Chickweed',
 'Common Chickweed',
 'Common Chickweed',
 'Common Chickweed',
 'Fat Hen',
 'Loose Silky-bent',
 'Common Chickweed',
 'Maize',
 'Common Chickweed',
 'Common Chickweed',
 'Common Chickweed',
 'Common Chickweed',
 'Common Chickweed',
 'Maize',
 'Common Chickweed',
 'Common Chickweed',
 'Maize',
 'Common Chickweed',
 'Common Chickweed',
 'Common Chickweed',
 'Loose Silky-bent',
 'Maize',
 'Common Chickweed',
 'Common Chickweed',
 'Common Chickweed',
 'Loose Silky-bent',
 'Common Chickweed',
 'Loose Silky-bent',
 'Common Chickweed',
 'Common Chickweed',
 'Common Chickweed',
 'Common Chickweed',
 'Fat Hen',
 'Common Chickweed',
 'Fat Hen',
 'Maize',
 'Common Chickweed',
 'Common Chickweed',
 'Maize',
 'Common Chickweed',
 'Common Chic

In [22]:
import pandas as pd

In [23]:
data_new=pd.DataFrame(ans,final)

In [25]:
data_new.head()

,0
f4caf74f9.png,Maize
0bf7bfb05.png,Loose Silky-bent
d2f422ccb.png,Common Chickweed
d0cdc768f.png,Loose Silky-bent
756dd5070.png,Common Chickweed


In [26]:
data_new.to_csv('submission_new.csv')